In [ ]:
# https://drive.google.com/drive/folders/1yN-6mGDY6XEmiUrZe2lu1NfRBiK2WjNJ
# https://drive.google.com/drive/folders/1hF2gyPKGfn4QzYfSnxgvyglr_v4kykNb
# s3://gamabhana/dict/raghuvir/english_to_hindi.tsv.gz

In [1]:
import pandas as pd
import numpy as np
import boto3

In [2]:
myheading = ['english', 'type' , 'hindi']

df = pd.read_csv('/home/ubuntu/raghuvir/final.tsv', sep = '\t', names = myheading)

In [3]:
df=df.dropna(subset = ['english'])

In [4]:
df=df.dropna(subset = ['hindi'])

In [5]:
df.english = df.english.str.lower()
df.type = df.type.str.lower()

In [6]:
threshold = 100
type_counts = df['type'].value_counts()

types_below_threshold = type_counts[type_counts < threshold].index

df.loc[df['type'].isin(types_below_threshold), 'type'] = ''

In [7]:
#df['extract'] = df['hindi'].str.extract('(\s[A-Za-z]+\.)', expand=True)
# df.extract.value_counts().to_dict()

In [8]:
mylist = {' m.': 'masculine',
 ' n.': 'noun common',
 ' f.': 'feminine',
 ' Ind.': 'Industrial',
 ' Chem.': 'chemistry',
 ' Bot.': 'botany',
 ' Zool.':  'zoology',
 ' Pharm.':  'pharmaceutical',
 ' I.':  'interjection',
 ' Astron.':  'Astronomy',
 ' adj.': 'Adjective',
 ' p.': 'plural',
 ' Anat.':  'Anatomy',
 ' Tax.': 'law',
 ' Math.':  'mathematics',
 ' Col.': 'colonial',
 ' Phys.':  'physics',
 ' Med.': 'medicine',
 ' N.': 'Noun Proper',
 ' Pathol.': 'Pathology',
 ' M.': 'masculine',
 ' Biol.':  'biology',
 ' Biochem.':  'Biochemistry',
 ' Gk.':  'General Knoledge',
 ' Mineral.':  'Mineral',
 ' Rly.': 'Railway',
 ' vb.': 'verb',
 ' Ecol.': 'ecology',
 ' Geol.': 'Geology',
 ' Eng.': 'engineering',
 ' Geog.': 'geography',
 ' Physiol.': 'Physiology',
 ' Mil.': 'military science',
 ' Econ.': 'economics',
 ' Metal.': 'Metal',
 ' Paleontol.': 'Paleontology',
 ' Engin.': 'engineering',
 ' Bact.':  'bacteriology',
 ' Psychol.': 'psychology',
 ' Admin.': 'administration',
 ' Educ.': 'education'
}

In [9]:
df['type2'] = ''

for i in mylist:
    mask = df['hindi'].str.contains(i, na=True)
    df.loc[mask, 'type2'] = mylist[i] + ' '

In [10]:
df = df.fillna('')

In [11]:
df = df.tail(30000)

In [12]:
df.to_csv('english_to_hindi2.tsv', sep='\t', index=False)

In [13]:
myl=df.T.to_dict().values()

In [14]:
# Connect to DynamoDB using boto

MY_ACCESS_KEY_ID = 'xxx'
MY_SECRET_ACCESS_KEY = 'xxx'

In [15]:
resource = boto3.resource('dynamodb', aws_access_key_id=MY_ACCESS_KEY_ID, 
                          aws_secret_access_key=MY_SECRET_ACCESS_KEY, region_name='us-east-1')

# Connect to the DynamoDB table

table = resource.Table('raghuvir4')

In [ ]:
# Load the JSON object created in the step 3 using put_item method

for student in myl:
    table.put_item(Item=student)